In [8]:
##############
# Minimizers #
##############

# Note: All minimizer functions returns an array L such that
# L[1] = x^*, or the optimal point
# L[2] = value of the objective function at the optimal point
# L[3] = number of iterations used or DEFAULT_MAX_ITER

const DEFAULT_MAX_ITER = 100
const DEFAULT_TOLERANCE = 0.01

# Minimization algorithm using gradient descent
# f  the objective function
# g  the gradient of f
# x  an initial testing point
# tolerance a small number
function minimize(f,g, x::Vector, tolerance = DEFAULT_TOLERANCE, max_iter = DEFAULT_MAX_ITER)
    alpha =0.3
    beta = 0.7
    iter = 0
    tolerance = tolerance*norm(g(x))
    while norm(g(x)) > tolerance && iter < max_iter
        dx = -g(x)
        t = 1
        while f(x+t*dx) > f(x) + alpha*t*sum(g(x).*dx)
            t = beta*t
        end
        x = x + t*dx
        iter = iter + 1
        print("$iter, ")
    end
    return [x, f(x), iter]
end


# Minimization algorithm using Newton's method
# f  the objective function
# g  the gradient of f
# h  the Hessian of f
# x  an initial testing point
# tolerance a small number
function minimizenewt(f,g, H, x::Vector, tolerance=DEFAULT_TOLERANCE, max_iter = DEFAULT_MAX_ITER)
    alpha =0.3
    beta = 0.7
    iter = 0
    
    # Default mode is RELATIVE tolerance, comment out the line below for ABSOLUTE tolerance
    tolerance = tolerance*norm(g(x))

    while norm(g(x)) > tolerance && iter < max_iter
        dx = -H(x)\g(x)
        t = 1
        while f(x+t*dx) > f(x) + alpha*t*sum(g(x).*dx)
            t = beta*t
        end
        x = x + t*dx
        iter = iter + 1
    end
    if iter < max_iter
        success =1
    else
        success =0
    end
    return [x, f(x), iter]
end




# Minimization algorithm using Newton's method, with fix for nonpositive definite matrix
# f  the objective function
# g  the gradient of f
# h  the Hessian of f
# x  an initial testing point
# tolerance a small number
function minimizenewtnpd(f,g, H, x::Vector, tolerance=DEFAULT_TOLERANCE, max_iter = DEFAULT_MAX_ITER)
    alpha =0.3
    beta = 0.7
    iter = 0
    function fixnonposdefmat(A, epsilon=0.1)
        M = eigfact(A)
        V = M[:vectors]
        L = M[:values]
        X = [max(abs(vi), epsilon)::Float64 for vi in L]
        L = diagm(X)
        return (V'*L*V)
    end
    
    # Default mode is RELATIVE tolerance, comment out the line below for ABSOLUTE tolerance
    tolerance = tolerance*sqrt(sum(g(x).^2))

    while sqrt(sum(g(x).^2)) > tolerance && iter < max_iter
        if isposdef(H(x))
            HH = H(x)
        else
            HH = fixnonposdefmat(H(x))
        end
        dx = -HH\g(x)
        t = 1
        while f(x+t*dx) > f(x) + alpha*t*sum(g(x).*dx)
            t = beta*t
        end
        x = x + t*dx
        iter = iter + 1
    end
    return [x, f(x), iter]
end




# Minimization algorithm using Newton's method, with fix for nonpositive definite matrix, version 2
# f  the objective function
# g  the gradient of f
# h  the Hessian of f
# x  an initial testing point
# tolerance a small number
function minimizenewtnpd2(f,g, H, x::Vector, tolerance=DEFAULT_TOLERANCE, max_iter = DEFAULT_MAX_ITER)
    alpha =0.3
    beta = 0.7
    iter = 0
    function fixnonposdefmat(A, epsilon=0.1)
        M = eigfact(A)
        V = M[:vectors]
        L = M[:values]
        X = [max(vi, epsilon)::Float64 for vi in L]
        L = diagm(X)
        return (V'*L*V)
    end
    
    # Default mode is RELATIVE tolerance, comment out the line below for ABSOLUTE tolerance
    tolerance = tolerance*norm(g(x))

    while norm(g(x)) > tolerance && iter < max_iter
        if isposdef(H(x))
            HH = H(x)
        else
            HH = fixnonposdefmat(H(x))
        end
        dx = -HH\g(x)
        t = 1
        while f(x+t*dx) > f(x) + alpha*t*sum(g(x).*dx)
            t = beta*t
        end
        x = x + t*dx
        iter = iter + 1
    end
    return [x, f(x), iter]
end







# Minimization algorithm using BFGS
# f  the objective function
# g  the gradient of f
# H  the initial guess of the INVERSE HESSIAN, you should set it to the identify matrix if you don't know
# x  an initial testing point
# tolerance a small number
function minimizeBFGS(f,g, H, x::Vector, tolerance=DEFAULT_TOLERANCE, max_iter = DEFAULT_MAX_ITER)
    alpha =0.3
    beta = 0.7
    iter = 0
    
    I = eye(length(x))
    
    # Default mode is RELATIVE tolerance, comment out the line below for ABSOLUTE tolerance
    tolerance = tolerance*sqrt(sum(g(x).^2))

    while norm(g(x)) > tolerance && iter < max_iter
        dx = -H*g(x)
        t = 1
        while f(x+t*dx) > f(x) + alpha*t*sum(g(x).*dx)
            t = beta*t
        end
        x1 = x + t*dx
        s = t*dx
        y = g(x1) - g(x)
        rho = 1/dot(s,y)
        H = (I- rho* s *(y'))*H*(I-rho*y*(s')) + rho*s*(s')
        x = x1
        iter = iter + 1
    end
    return [x, f(x), iter]
end




minimizeBFGS (generic function with 3 methods)

In [9]:
################################
# Simple test with a quadratic #
################################

m = 100
n = 2
x0 = [3;1]
A = randn(m,n)
b = A*x0
f0(x) = 0.5*norm(A*x-b)^2
g0(x) = A'*(A*x-b)
H0(x) = A'*A

println("Legends:")
println("NM = Newton's method")
println("NMPDF1 = Newton's method with non-positive definite Hessian fix, WITH absolute value")
println("NMPDF2 = Newton's method with non-positive definite Hessian fix, WITHOUT absolute value")
println("BFGS = quasi-Newton method using BFGS approximation of inverse Hessian")
println("Correct = the correct outputs a minimizer function should spit out")
println("---------------------------------------------------------------------------------------")
@printf("%-10s %-40s %20s %20s\n", "Method", "Coordinate of optimal point", "Value of minimum", "# of iterations")
L = minimizenewt(f0,g0,H0,[0;0])
@printf("%-10s %10f %20f %20f %20i\n", "NM", L[1], L[2], L[3], L[4])
L = minimizenewtnpd(f0,g0,H0,[0;0])
@printf("%-10s %10f %20f %20f %20i\n", "NMPDF1", L[1], L[2], L[3], L[4])
L = minimizenewtnpd2(f0,g0,H0,[0;0])
@printf("%-10s %10f %20f %20f %20i\n", "NMPDF2", L[1], L[2], L[3], L[4])
L = minimizeBFGS(f0,g0,eye(length(x0)), [0;0])
@printf("%-10s %10f %20f %20f %20i\n", "BFGS", L[1], L[2], L[3], L[4])
@printf("%-10s %10f %20f %20f %20i\n", "Correct", 3.0, 1.0, 0.0, 0)




Legends:
NM = Newton's method
NMPDF1 = Newton's method with non-positive definite Hessian fix, WITH absolute value
NMPDF2 = Newton's method with non-positive definite Hessian fix, WITHOUT absolute value
BFGS = quasi-Newton method using BFGS approximation of inverse Hessian
Correct = the correct outputs a minimizer function should spit out
---------------------------------------------------------------------------------------
Method     Coordinate of optimal point                  Value of minimum      # of iterations
NM           3.000000             1.000000             0.000000                    1
NMPDF1       3.000000             1.000000             0.000000                    1
NMPDF2       3.000000             1.000000             0.000000                    1
BFGS         3.000009             0.999972             0.000000                    4
Correct      3.000000             1.000000             0.000000                    0


In [10]:
Pkg.add("Toms566")
Pkg.update()
using Toms566

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of Toms566
INFO: Use `Pkg.update()` to get the latest versions of your packages
INFO: Updating METADATA...
INFO: Updating Toms566...
INFO: Computing changes...
INFO: No packages to install, update or remove


In [11]:

######################################################
# Complete test of every functions in Toms566, SLOW! #
######################################################

println("Legends:")
println("NM = Newton's method")
println("NMPDF1 = Newton's method with non-positive definite Hessian fix, WITH absolute value")
println("NMPDF2 = Newton's method with non-positive definite Hessian fix, WITHOUT absolute value")
println("BFGS = quasi-Newton method using BFGS approximation of inverse Hessian")
println("---------------------------------------------------------------------------------------")
@printf("%3s  %-30s  %-30s", "", "", "# of iterations (-1 = failure)\n")
@printf("%3s  %-30s  %-6s %-8s %-8s %-8s\n",
        "No.",
        "Name",
        "NM", 
        "NMPDF1",
        "NMPDF2",
        "BFGS")
for i=1:18
    p = Problem(i)
    x0 = p.x0       # standard starting point
    f = p.obj   # objective value at x
    g = p.grd   # gradient at x
    H = p.hes   # Hessian at x
    
    # Keeps track of the number of iterations
    iters = [minimizenewt(f,g,H,x0)[end], 
            minimizenewtnpd(f,g,H,x0)[end],
            minimizenewtnpd2(f,g,H,x0)[end],
            minimizeBFGS(f,g,eye(length(x0)), x0)[end]]
    
    # Set the number of iterations to be X if there was an exceeding of max allowed number of iterations
    function showFailure(iter)
        if iter == DEFAULT_MAX_ITER
            return -1
        else
            return iter
        end
    end
    iters = map(showFailure, iters)

    # NM = Newton's method
    # NMNPDF = Newton's method with non-positive definite fix
    @printf("%3i  %-31s %-6i %-8i %-8i %-8i\n",
            i, 
            p.name,
            iters[1],
            iters[2],
            iters[3],
            iters[4])
end

Legends:
NM = Newton's method
NMPDF1 = Newton's method with non-positive definite Hessian fix, WITH absolute value
NMPDF2 = Newton's method with non-positive definite Hessian fix, WITHOUT absolute value
BFGS = quasi-Newton method using BFGS approximation of inverse Hessian
---------------------------------------------------------------------------------------
                                     # of iterations (-1 = failure)
No.  Name                            NM     NMPDF1   NMPDF2   BFGS    
  1  Hellical valley                 1      4        10       6       
  2  Bigg's EXP6                     -1     -1       -1       15      
  3  Gaussian                        1      1        1        2       
  4  Powell                          -1     12       3        8       
  5  Box 3-dim                       2      3        6        7       
  6  Variably dimensioned            6      2        2        1       
  7  Watson                          -1     31       -1       4       
  